<a href="https://colab.research.google.com/github/MattIzon/16010269_DataAnalytics/blob/main/1_Crime_Aquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set-up
from google.colab import auth, files
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)
import pandas as pd
pd.set_option('display.max_columns', None)

Authenticated


In [2]:
# Use big query to get the crime data
client = bigquery.Client(project='assignment-305921')

crime = []

for year in range(2016, 2021):
  crime.append(client.query('''
  SELECT *
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = {}
  ORDER BY date
  '''.format(year)).to_dataframe())

In [12]:
# inspect the data and decide:
# a) Which columns may be useful
# b) Which columns need prep before they can be used - missing data, wrong format etc
crime[0].head()

,date,iucr,primary_type,location_description,domestic,beat,district,ward,community_area,latitude,longitude
0,2016-01-01 01:00:00+00:00,0620,BURGLARY,APARTMENT,False,1923,19,44,6,41.949534,-87.661660
1,2016-01-01 01:00:00+00:00,0820,THEFT,CTA TRAIN,False,1114,11,28,26,41.885488,-87.726422
2,2016-01-01 01:00:00+00:00,0820,THEFT,STREET,False,1222,12,2,27,41.882914,-87.694674
3,2016-01-01 01:00:00+00:00,1320,CRIMINAL DAMAGE,STREET,False,332,3,5,43,41.768906,-87.577604
4,2016-01-01 01:00:00+00:00,2820,OTHER OFFENSE,RESIDENCE,False,724,7,17,68,41.773529,-87.651911


*   At first glance: unique_key, case_number, block, description, arrest, fbi_code, x_coordinate, y_coordinate, year, updated_on dont appear to have any bearing on crime numbers.

*   date requires reformatting

*   Types of crime could increase on certain days.
iucr seems to be a reduction of primary_type and description into a single field.
This needs investigating and decision made as to which type of categorisation is more useful.

*   location_description could show crime increasing in certain locations on given days

*   domestic crime could increase on weekends which would impact crime rates on particular days



*   beat, district, ward and community_area provide different levels of location boundary to a crime. 
Crime patterns could emerge in specific locals. A choice of boundary level needs to be made.

*   latitude and longitude / location provide a means of connection with weather data.
location is a reduction of lat / lon. If data is missing one may be used to fill the other.

In [4]:
# Drop the unnecessary fields
no_use = ['unique_key', 'case_number', 'block', 'description', 'arrest', 'fbi_code', 'x_coordinate', 'y_coordinate', 'year', 'updated_on']
for year in crime:
  year.drop(no_use, axis=1, inplace=True)

In [5]:
# Check for missing data
index = 0
for year in crime:
  print('Missing Data ', 2016 + index, ':')
  for column in year:
    print('  ', column, ': ', year[column].isna().any())
  index += 1
  print()

Missing Data  2016 :
   date :  False
   iucr :  False
   primary_type :  False
   location_description :  True
   domestic :  False
   beat :  False
   district :  False
   ward :  False
   community_area :  False
   latitude :  True
   longitude :  True
   location :  True

Missing Data  2017 :
   date :  False
   iucr :  False
   primary_type :  False
   location_description :  True
   domestic :  False
   beat :  False
   district :  True
   ward :  True
   community_area :  False
   latitude :  True
   longitude :  True
   location :  True

Missing Data  2018 :
   date :  False
   iucr :  False
   primary_type :  False
   location_description :  True
   domestic :  False
   beat :  False
   district :  False
   ward :  True
   community_area :  False
   latitude :  True
   longitude :  True
   location :  True

Missing Data  2019 :
   date :  False
   iucr :  False
   primary_type :  False
   location_description :  True
   domestic :  False
   beat :  False
   district :  False
 



*   location_description, district (2017), ward (2017/2018/2019/2020), community_area (2020), latitude, longitude and location have missing data

*   district, ward and community_area are linked

*   latitude, longitude and location are linked

In [6]:
# Inspect missing data
index = 0
for year in crime:
  print(2016 + index, ':')
  total_rows = year.shape[0]
  print('Total rows: ', total_rows)
  missing_columns = ['location_description', 'district', 'ward', 'community_area', 'latitude', 'longitude', 'location']
  for column in missing_columns:
    missing_count = sum(year[column].isna())
    print(column, ' missing count: ', missing_count, ' - Percentage of total: ', (missing_count/total_rows)*100)
  index += 1
  print()

2016 :
Total rows:  269706
location_description  missing count:  1276  - Percentage of total:  0.47310775436957275
district  missing count:  0  - Percentage of total:  0.0
ward  missing count:  0  - Percentage of total:  0.0
community_area  missing count:  0  - Percentage of total:  0.0
latitude  missing count:  2451  - Percentage of total:  0.908767324419924
longitude  missing count:  2451  - Percentage of total:  0.908767324419924
location  missing count:  2451  - Percentage of total:  0.908767324419924

2017 :
Total rows:  268953
location_description  missing count:  1276  - Percentage of total:  0.47443233576126687
district  missing count:  1  - Percentage of total:  0.0003718121753614944
ward  missing count:  1  - Percentage of total:  0.0003718121753614944
community_area  missing count:  0  - Percentage of total:  0.0
latitude  missing count:  4007  - Percentage of total:  1.489851386673508
longitude  missing count:  4007  - Percentage of total:  1.489851386673508
location  missi

*   latitude, longitude and location have equal numbers of missing data. If the missing items are all in the same rows the location column is not required.

*   Potentially district, ward and community_area can be patched using each other's values. Further inspection to determine their usefulness should be completed first.

In [7]:
# Check if latitude, longitude and location's missing data are all in the same rows.
index = 0
for year in crime:
  print(sum(year['latitude'].isna() & year['longitude'].isna() & year['location'].isna()))
  index += 1
  print()

2451

4007

5146

1894

3292



* Figures match for each year, therefore all missing from the same row. Location column can be dropped.

In [8]:
# Drop the location field
no_use = ['location']
for year in crime:
  year.drop(no_use, axis=1, inplace=True)

In [10]:
crime[0].head()

,date,iucr,primary_type,location_description,domestic,beat,district,ward,community_area,latitude,longitude
0,2016-01-01 01:00:00+00:00,0620,BURGLARY,APARTMENT,False,1923,19,44,6,41.949534,-87.661660
1,2016-01-01 01:00:00+00:00,0820,THEFT,CTA TRAIN,False,1114,11,28,26,41.885488,-87.726422
2,2016-01-01 01:00:00+00:00,0820,THEFT,STREET,False,1222,12,2,27,41.882914,-87.694674
3,2016-01-01 01:00:00+00:00,1320,CRIMINAL DAMAGE,STREET,False,332,3,5,43,41.768906,-87.577604
4,2016-01-01 01:00:00+00:00,2820,OTHER OFFENSE,RESIDENCE,False,724,7,17,68,41.773529,-87.651911


In [13]:
# Inspect beat, district, ward, community_area
index = 0
for year in crime:
  print(2016 + index, ':')
  columns = ['beat', 'district', 'ward', 'community_area']
  for column in columns:
    unique_count = year[column].nunique()
    print(column, ' unique count: ', unique_count)
  index += 1
  print()

2016 :
beat  unique count:  274
district  unique count:  23
ward  unique count:  50
community_area  unique count:  77

2017 :
beat  unique count:  274
district  unique count:  23
ward  unique count:  50
community_area  unique count:  77

2018 :
beat  unique count:  274
district  unique count:  23
ward  unique count:  50
community_area  unique count:  77

2019 :
beat  unique count:  274
district  unique count:  23
ward  unique count:  50
community_area  unique count:  77

2020 :
beat  unique count:  274
district  unique count:  23
ward  unique count:  50
community_area  unique count:  77



* Identical figures for each year suggest these are the total numbers of each beat, district, ward and community_area

* beat has the largest number of unique values (274) and so must be the narrowest area.
As the smallest area, the average latitude / longitude per beat is the best solution available to fill the missing data in those columns.

* district has the lowest number of unique values (23) and so must be the widest area.
This could be useful for the DNN, however the missing row in 2017 will need addressing 

In [17]:
# Fill missing latitude / longitude data
for year in crime:
  beats = year['beat'].unique().tolist()

  for beat in beats:
    year.loc[(year['beat'] == beat) & (year['latitude'].isnull()), 'latitude'] = year[year['beat'] == beat]['latitude'].mean(axis = 0, skipna = True)
    year.loc[(year['beat'] == beat) & (year['longitude'].isnull()), 'longitude'] = year[year['beat'] == beat]['longitude'].mean(axis = 0, skipna = True)

In [23]:
# Convert the datetime to date column
for year in crime:
  year['date'] = pd.to_datetime(year['date']).dt.date

In [24]:
crime[0].head()

,date,iucr,primary_type,location_description,domestic,beat,district,ward,community_area,latitude,longitude
0,2016-01-01,0620,BURGLARY,APARTMENT,False,1923,19,44,6,41.949534,-87.661660
1,2016-01-01,0820,THEFT,CTA TRAIN,False,1114,11,28,26,41.885488,-87.726422
2,2016-01-01,0820,THEFT,STREET,False,1222,12,2,27,41.882914,-87.694674
3,2016-01-01,1320,CRIMINAL DAMAGE,STREET,False,332,3,5,43,41.768906,-87.577604
4,2016-01-01,2820,OTHER OFFENSE,RESIDENCE,False,724,7,17,68,41.773529,-87.651911


In [26]:
# Check iucr and primary_type for unique records
index = 0
print('Unique Data:')
for year in crime:
  print(2016 + index, ':')
  print('iucr: ', len(year['iucr'].unique()))
  print('primary_type: ', len(year['primary_type'].unique()))
  index += 1
  print()

Unique Data:
2016 :
iucr:  326
primary_type:  34

2017 :
iucr:  332
primary_type:  33

2018 :
iucr:  323
primary_type:  33

2019 :
iucr:  319
primary_type:  32

2020 :
iucr:  313
primary_type:  33



*   I believe primary_type would be more useful as it has fewer categories and looks at crime types from a higher level.

*   I think iucr's 300+ categories could dilute the data too much for Linear Regression, however it may prove useful to the DNN?

*   Have two datasets with only one in each?


In [ ]:
crime[0]['counts'] = crime[0].location_description.map(crime[0].location_description.value_counts())
print(crime[0])

              date  iucr     primary_type location_description  domestic  \
0       2016-01-01  0820            THEFT            RESIDENCE     False   
1       2016-01-01  0460          BATTERY            CTA TRAIN     False   
2       2016-01-01  0870            THEFT  TAVERN/LIQUOR STORE     False   
3       2016-01-01  1320  CRIMINAL DAMAGE               STREET     False   
4       2016-01-01  1562      SEX OFFENSE            APARTMENT      True   
...            ...   ...              ...                  ...       ...   
269701  2016-12-31  2820    OTHER OFFENSE            RESIDENCE     False   
269702  2016-12-31  0810            THEFT            CTA TRAIN     False   
269703  2016-12-31  0486          BATTERY            APARTMENT      True   
269704  2016-12-31  041A          BATTERY    CONVENIENCE STORE     False   
269705  2016-12-31  0460          BATTERY        BAR OR TAVERN     False   

        beat  district  ward  community_area   latitude  longitude  \
0       1921     

In [ ]:
# Save crime data to CSV files.

names = ['bq_2016', 'bq_2017', 'bq_2018', 'bq_2019', 'bq_2020']

# for index in range(len(names)):
#   with open('/gdrive/My Drive/assignment/data/crime/{}.csv'.format(names[index]), 'w') as f:
#     crime[index].to_csv(f)

for index in range(len(names)):
  crime[index].to_csv('{}.csv'.format(names[index])) 
  files.download('{}.csv'.format(names[index]))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>